In [1]:
!nvidia-smi

Sat Apr  5 17:22:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.60                 Driver Version: 572.60         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   53C    P0              9W /  114W |     903MiB /   8188MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# purpose of accelerate: 
1. Ease of Multi-Device Training: Whether you're using multiple GPUs or TPUs, accelerate makes it easier to scale your model accross devices with minimal code
2. Mixed percision: it allows models to be trained using mixed precision, which can speed up training and reduce memory useage
3. Zero Redundancy Optimizer (ZeRO): Helps manage large models be efficiently splitting the model across multiple devices. Offiad to CPU/SSD: Useful for large models that may not fit entirely into GPU memory, by allowing parts of the model or optimizer to be offloaded tp CPU or even SSD

In [2]:
! pip install --upgrade accelerate
! pip install -y transformer accelerate
! pip install transformers accelerate


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [3]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt 
from datasets import load_dataset
import pandas as pd 
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm 
import torch
nltk.download('punkt')

d:\pythonProjects\Agentic_AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\26amr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Basic Functionality of Hugging Face model

In [4]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration

model=PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
tokenizer=AutoTokenizer.from_pretrained('google/pegasus-xsum')

ARTICLE_TO_SUMMERIZE=(
    "Crowds of people angry about the way President Donald Trump is running the country marched and rallied in scores of American cities Saturday in the biggest day of demonstrations yet by an opposition movement trying to regain its momentum after the shock of the Republican’s first weeks in office."
    "So-called Hands Off! demonstrations were organized for more than 1,200 locations in all 50 states by more than 150 groups including civil rights organizations, labor unions, LBGTQ+ advocates, veterans and elections activists. The rallies appeared peaceful, with no immediate reports of arrests."
    "From the National Mall and Midtown Manhattan to Boston Common and multiple state capitols, thousands of protesters assailed Trump and billionaire Elon Musk ‘s actions on government downsizing, the economy, immigration and human rights. In Seattle, in the shadow of the city’s iconic Space Needle, protesters held signs with slogans like “Fight the oligarchy."
)
inputs=tokenizer(ARTICLE_TO_SUMMERIZE, max_length=1024,return_tensors='pt',truncation=True)

# generate summary 
summary_ids = model.generate(inputs['input_ids'])
tokenizer.batch_decode(summary_ids,skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'.'

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# fine Tuning

In [8]:
model='google/pegasus-cnn_dailymail'
tokenizer=AutoTokenizer.from_pretrained(model)
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# download & unzip data
import os 
import zipfile
os.chdir('../')
def extract_unzip_file():
    unzip_path='dataset'
    os.makedirs(unzip_path,exist_ok=True)
    with zipfile.ZipFile('dataset/summarizer-data.zip','r') as zip_ref:
        zip_ref.extractall(unzip_path)

extract_unzip_file()


In [21]:
dataset_samsun=load_from_disk('dataset/samsum_dataset')
dataset_samsun

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})